In [87]:
from graph_tool.all import load_graph, shortest_distance
from ic import observe_cascade, simulate_cascade, sample_graph_by_p, MAXINT
from tqdm import tqdm
from joblib import Parallel, delayed

In [88]:
g = load_graph('data/arxiv-hep-th/graph.graphml')

In [89]:
print(g.num_vertices())

8638


In [99]:
p = 0.7
source, cascade = simulate_cascade(g, p)

In [100]:
print(np.sum(cascade != -1))
print(g.num_vertices())

7644
8638


In [105]:
obs = observe_cascade(cascade, q=0.01, method='uniform')

In [106]:
print(len(obs))

77


In [110]:
counts = np.zeros(g.num_vertices())
gv = sample_graph_by_p(g, p)
source_times = np.zeros((len(obs), g.num_vertices()))
for i, o in enumerate(obs):
    c = shortest_distance(gv, source=gv.vertex(o)).a
    inactive_nodes = np.nonzero(c == MAXINT)[0]
    source_times[i, :] = cascade[o] - c
    source_times[i, :][inactive_nodes] = cascade[o] + 1
identity = np.tile(source_times[0], len(obs)).reshape(g.num_vertices(), len(obs))
np.all(identity == source_times, axis=0)

/home/cloud-user/code/source_finding/venv/lib/python3.5/site-packages/ipykernel/__main__.py:10: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.


False

In [133]:
source_times.mean(axis=0)[source]

0.18181818181818182

In [132]:
print(np.sort(np.abs(source_times.mean(axis=0)))[:100])
# np.median(source_times, axis=0)

[ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701
  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701
  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701
  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701  0.01298701
  0.01298701  0.01298701  0.02597403  0.02597403  0.02597403  0.02597403
  0.02597403  0.02597403  0.02597403  0.02597403  0.02597403  0.02597403
  0.02597403  0.02597403  0.02597403  0.02597403  0.02597403  0.02597403
  0.02597403  0.02597403  0.02597403  0.02597403  0.02597403  0.02597403
  0.03896104  0.03896104  0.03896104  0.03896104  0.03896104  0.03896104
  0.03896104  0.03896104  0.03896104  0.03896104  0.03896104  0.03896104
  0.03896104  0.03896104  0.03896104  0.03896104  0.03896104  0.03896104
  0.03896104  0.03896104  0.03896104  0.03896104  0.038

In [74]:
def estimate_source_likelihood(g, obs, cascade, p):
    counts = np.zeros(g.num_vertices())
    gv = sample_graph_by_p(g, p)
    source_times = np.zeros((len(obs), g.num_vertices()))
    for i, o in enumerate(obs):
        c = shortest_distance(gv, source=gv.vertex(o)).a
        c[c == MAXINT] = -1        
        active_nodes = np.nonzero(c != -1)[0]
        source_times[i, :] = cascade[o] - c
    identity = np.tile(source_times[0], len(obs)).reshape(g.num_vertices(), len(obs))
    np.all(identity == source_times, axis=0)
    
    counts[possible_sources] += 1
    return counts


In [75]:
K = 8

counts_list = Parallel(n_jobs=-1)(delayed(estimate_source_likelihood)(g, obs, cascade, p)
                                  for _ in tqdm(range(K)))

100%|██████████| 8/8 [00:00<00:00, 20.93it/s]


In [76]:
counts = np.array(counts_list).sum(axis=0)
print(counts.shape)

(8638,)


In [77]:
print(counts[source], counts.max())
print(np.sum(counts == counts.max()))

8.0 43672.0
6496
